In [152]:
from scraper_functions import *
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime
import time
from random import random
import pickle
from selenium.webdriver.common.by import By
from io import StringIO
def js_click(element):
    driver.execute_script('arguments[0].click();', element)
from glob import glob
from tqdm import tqdm
import os
import re
import requests
from bs4 import BeautifulSoup


Done, the environment is ready. Now Teach Me Scrape!



In [153]:
def process_table_row(table_row):
    _, popularity_str, table_name_str, universe_str, classifications_str, year_dataset_str, breakdowns_str = [td.get_attribute('outerHTML') for td in table_row.find_elements(By.TAG_NAME, 'td')]
    popularity = int(re.findall(r'"width\:(\d+)\%\;"',popularity_str)[-1])
    table_label = re.findall(r'<span class="monospace">(.*?)\.</span>',table_name_str)[0]
    table_name = re.findall(r'<span class="grid-table-lable">(.*?)</span>',table_name_str)[0]
    universe = re.sub(r'<.*?>','',universe_str).strip()
    classifications = tuple(re.findall(r'<a class="classification-details" href="#" id=".*?">(.*?)</a>',classifications_str))
    year_dataset = re.sub(r'<.*?>','',year_dataset_str).strip()
    breakdowns = re.sub(r'<.*?>','',breakdowns_str).strip()
    return table_label, table_name, universe, classifications, year_dataset, breakdowns, popularity

#### TIME SERIES TABLE (TRACT LEVEL, ACS 2006-2022)

In [189]:
# tract_ts_df = pd.read_html(StringIO(driver.find_element(By.XPATH, '//*[@id="time-series-grid"]').get_attribute('outerHTML')))[0].loc[1:,2:].drop(4,axis=1)
# tract_ts_df.columns = ['sequence','table_name','years','geographic_levels']
# tract_ts_df = tract_ts_df.reset_index(drop=True)
# tract_ts_df.to_feather(X 'nhgis_selected_time_series_tables.feather')

#### SOURCE TABLE (TRACT, BLOCKGROUP, BLOCK LEVELS)

In [168]:
# data_list = []

In [169]:
# table = driver.find_element(By.ID, 'source-tables-grid')
# table_rows = [tr for tr in table.find_elements(By.TAG_NAME,'tr')][1:]
# data_list += [process_table_row(table_row) for table_row in table_rows]
# len(data_list)//500

In [36]:
# tract_df = pd.DataFrame(data_list, columns='table_label, table_name, universe, classifications, year_dataset, breakdowns, popularity'.split(', '))
# tract_df.to_feather('tract_df.feather')

In [56]:
# blockgroup_df = pd.DataFrame(data_list, columns='table_label, table_name, universe, classifications, year_dataset, breakdowns, popularity'.split(', '))
# blockgroup_df.to_feather('blockgroup_df.feather')

In [61]:
# block_df = pd.DataFrame(data_list, columns='table_label, table_name, universe, classifications, year_dataset, breakdowns, popularity'.split(', '))
# block_df.to_feather('block_df.feather')

In [4]:
import pandas as pd
import numpy as np
import re
from collections import Counter
def flatten_list(l):
    return [item for sublist in l for item in sublist] 

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)
pd.set_option('chained_assignment', None)

tract_df = pd.read_feather('tract_df.feather')
tract_df = tract_df.drop('breakdowns', axis=1)
tract_df['geographic_level'] = 'tract'

blockgroup_df = pd.read_feather('blockgroup_df.feather')
blockgroup_df = blockgroup_df.drop('breakdowns', axis=1)
blockgroup_df['geographic_level'] = 'blockgroup'

block_df = pd.read_feather('block_df.feather')
block_df = block_df.drop('breakdowns', axis=1)
block_df['geographic_level'] = 'block'

nhgis_df = pd.concat([tract_df, blockgroup_df, block_df], ignore_index=True)
nhgis_df['year'] = nhgis_df['year_dataset'].apply(lambda x: [n for n in x.split('_') if n.isnumeric()][-1]).astype(int)

# Counter(flatten_list(nhgis_df['classifications'])).most_common()
# nhgis_df.loc[(nhgis_df['classifications'].apply(lambda li: 'Imputation/Allocation (2)' in li))&(nhgis_df['table_name'].apply(lambda x: ' '.join(x.split()[:2]) == 'Age by'))]
nhgis_df.loc[nhgis_df['classifications'].apply(lambda li: any(['Imputation/Allocation ' in x for x in li])),'imputed_or_allocated'] = True
nhgis_df['imputed_or_allocated'] = nhgis_df['imputed_or_allocated'].fillna(False)

nhgis_df.loc[nhgis_df['table_label'].str.endswith('PR'),'puerto_rico_only'] = True
nhgis_df['puerto_rico_only'] = nhgis_df['puerto_rico_only'].fillna(False)

nhgis_df['subgroup_alone'] = nhgis_df['table_label'].apply(lambda x: x[-1].isalpha())
nhgis_df['subgroup_alone'] = nhgis_df['subgroup_alone'].fillna(False)

nhgis_df['table_category'] = nhgis_df['table_label'].apply(lambda x: x[0])
# nhgis_df['table_category'].value_counts()
# B    17545
# C     1438
# N      521
# P      268
# H      114

nhgis_df.to_feather(X 'nhgis_selected_source_tables.feather')